In [1]:
# %load guba_crawl_topic.py
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 23 2018

@author: zhangjoy
爬取东方财富网股吧-热门主题吧
数据文件采用csv存取
"""

import requests
from bs4 import BeautifulSoup
import csv
import time

# 获取页面内容
def getHTMLText(url):
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        return ""

# 这里比较m_time是否大于n_time
def compare_time(m_time,n_time):
    m_time1 = time.mktime(time.strptime(m_time,'%Y-%m-%d %H:%M:%S')) 
    n_time1 = time.mktime(time.strptime(n_time,'%Y%m%d'))
    if (float(m_time1) >= float(n_time1)):
        return 1
    else:
        return 0

# 爬取热门主题吧：主题 + 网页链接
'''
    财经评论
    http://guba.eastmoney.com/list,cjpl,99_1.html
    http://guba.eastmoney.com/list,cjpl,99_2.html ...
'''
def hot_zhutiba(topic_list):
    html = getHTMLText("http://guba.eastmoney.com/remenba.aspx?type=2")    #该网址为股吧热门主题吧列表
    soup = BeautifulSoup(html,'html.parser') # 解析网页
    a1 = soup.find("div",attrs={"class":"zhutibarlist"})    #body > div:nth-child(6) > div.gbboxb > div > ul > ul > li:nth-child(1) > a
    a2 = a1.find_all("a")                                   #热门主题吧："class":"zhutibarlist"
    one = []                                                #全部主题吧："class":"allzhutilistb"
    for tr in a2:
        title = tr.text # 主题吧 名称
        href = tr.attrs["href"] # list,cjpl.html
        href = href.split('.')[0].split(",")[1] # 主题吧热贴 http://guba.eastmoney.com/list,cjpl,99.html
        one.append(title)                       #
        one.append(href)
        topic_list.append(one)
        # file = "./py_project/stock_crawl_new/topic.csv"
        # with open(file, 'a', newline="") as f:
        #     #dialect为打开csv文件的方式，默认是excel，delimiter="\t"参数指写入的时候的分隔符
        #     csvwriter = csv.writer(f,dialect = ("excel"))
        #     #csv文件插入一行数据，把下面列表中的每一项放入一个单元格（可以用循环插入多行）
        #     csvwriter.writerow(one)
        one = []
    return topic_list
'''
[[['财经评论吧', 'cjpl'], ['股市实战吧', 'gssz'], ['上证指数吧', 'szzs'], ['玉米吧', 'fdcec'], 
 ['螺纹钢吧', 'fshferb'], ['铁矿石吧', 'fdcei'], ['房地产吧', 'fdc'], ['焦炭主力吧', 'fdcejm'], 
 ['股民学校吧', 'gmxx'], ['新三板吧', 'bk0600'], ['沪金吧', 'fshfeau'], ['焦炭吧', 'fdcej'], 
 ['美元指数吧', 'zsgjudi'], ['核心内参吧', 'hxnc'], ['股指期货吧', 'gzqh'], ['创业板指吧', 'cyb'], 
 ['甲醇吧', 'fczcema'], ['橡胶吧', 'fshferu'], ['原油吧', 'yuanyou'], ['恒生指数吧', 'hkhsi'], 
 ['现货吧', 'xianhuo'], ['大话财经吧', 'dhcj'], ['新股吧', 'xg'], ['玻璃吧', 'fczcefg'], 
 ['豆粕吧', 'fdcem'], ['理财吧', 'licai'], ['白糖单吧', 'fczcesr'], ['PTA吧', 'fczceta'], 
  ['沪银吧', 'fshfeag'], ['期货吧', 'qh']]]
'''

# 获取评论的发表时间，内容
def getCommentContent(url):
    publish_time,content = "",""
    html = getHTMLText(url) # [url]"http://guba.eastmoney.com/news,cjpl,746982237.html"
    soup = BeautifulSoup(html,'html.parser')
    # title = soup.title.text.split("_")[0] # 标题
    a = soup.find("div",attrs={"id":"zwconbody"})
    # a1 = a.find("div",attrs={"id":"zw_header"})
    # come_from = a1.text.split(" ")[0].split("：")[1] # 来源
    # b1 = a.find("div",attrs={"id":"zw_body"})
    # b = a.find_all("p")
    # for tr in b:
    #     content = content + tr.text # 内容
    # content = a.text
    if a != None:
        try:      
            for string in a.stripped_strings:
                content = content + string
        except:        
            content = content + a.string
    else:
        content = "wrong"
    c = soup.find("div",attrs={"class":"zwfbtime"})
    try:
        publish_time = c.text.split(" ")[1] + " " + c.text.split(" ")[2] # 发表时间 发表于 2018-02-24 08:42:00
    except:
        publish_time = c.text.split(" ")[0].split("：")[1] + " " + c.text.split(" ")[1] # 发表于：2018-02-26 10:53:04
    return publish_time,content

# 获取评论信息
def getCommentInfo(start_url,file,page,start,end,topic_bar,topic_url):
    flag = 1
    html = getHTMLText(start_url) # 热帖开始网址 http://guba.eastmoney.com/list,cjpl,99.html
    soup = BeautifulSoup(html,'html.parser')
    # header
    # a1 = soup.find("div",attrs={"class":"dheader"})
    # header_list = [] # ['阅读', '评论', '标题', '作者', '发表日期', '最后更新']
    # a2 = a1.find_all("span")
    # for tr in a2:
    #     header = tr.text
    #     header_list.append(header)
    # # 获取帖子['阅读', '评论', '标题', '作者', '发表日期', '最后更新']
    Oneinfo = []
    count = 0
    b1 = soup.find_all("div",attrs={"class":"articleh"})
    if b1 != None:
        for tr in b1:
            t1 = time.time()
            count = count + 1
            read_num = tr.find("span",attrs={"class":"l1"}).text
            comment_num = tr.find("span",attrs={"class":"l2"}).text
            title = tr.find("span",attrs={"class":"l3"}).text
            content_url = "http://guba.eastmoney.com" + tr.find("span",attrs={"class":"l3"}).a.attrs["href"]
            publish_time,content = getCommentContent(content_url)
            author = tr.find("span",attrs={"class":"l4"}).text
            # publish_time = tr.find("span",attrs={"class":"l5"}).text # 没有年份
            # latest_time = tr.find("span",attrs={"class":"l6"}).text

            # 根据发帖时间判断是否需要爬取下一页以及是否需要爬取帖子标题和内容
            time_fit1 = compare_time(publish_time,start) # > :1 晚 else:0 早
            time_fit2 = compare_time(publish_time,end)   # > :1 晚 else:0 早

            # 当前帖子的发帖时间小于爬取开始时间，不需要继续爬取下一页评论列表，设置flag=0
            if time_fit1 == 0:
                flag = 0
            else:
                flag = 1

            #当前帖子的发帖时间处于爬取时间段内，对帖子标题和内容进行爬取
            if time_fit1 == 1 and time_fit2 == 0:
                Oneinfo.append(["page_"+str(page),count]) # 页面信息
                Oneinfo.append(title) # 评论标题
                Oneinfo.append(author) # 发表作者
                Oneinfo.append(publish_time) # 发表时间
                Oneinfo.append(read_num) # 阅读量
                Oneinfo.append(comment_num) # 评论量
                Oneinfo.append(content_url) # 内容链接: http://guba.eastmoney.com/news,cjpl,743193430.html
                Oneinfo.append(content) # 评论内容
                # Oneinfo.append(latest_time) # 最后更新时间
                with open(file, 'a', newline="", encoding='gb18030') as f: # ,encoding="gb18030"
                    # dialect为打开csv文件的方式，默认是excel，delimiter="\t"参数指写入的时候的分隔符
                    csvwriter = csv.writer(f,dialect = ("excel"))
                    # csv文件插入一行数据，把下面列表中的每一项放入一个单元格（可以用循环插入多行）
                    csvwriter.writerow(Oneinfo)
                t2 = time.time()
                print(topic_bar,"p"+str(page),count,"In and Crawling success,time using %.5f s" %(t2-t1))
                Oneinfo = []
            else:
                print(topic_bar,"p"+str(page),count,"Out")
    else:
        flag = 0

    if flag == 1:
        return "http://guba.eastmoney.com/list," + topic_url + ",99,f_" + str(page+1) + ".html"  
    else:
        return ""

    '''                 
    <span class="l1">14462545</span>
    <span class="l2">231</span>
    <span class="l3"><a href="/news,cjpl,743193430.html" title="1分钟股票开户立享多重惊喜：超低佣金，另有5大豪礼等你拿！" >1分钟股票开户立享多重惊喜：超低佣金，另有5大</a></span>
    <span class="l4"><a href="http://iguba.eastmoney.com/6712111507146464"  data-popper="6712111507146464" data-poptype="1" target="_blank">东方财富网</a><input type="hidden" value="31000" /><a href=" http://v2.eastmoney.com"><em class="lv" title="东方财富机构认证"></em></a></span>
    <span class="l6">01-25</span>
    <span class="l5">02-24 12:12</span>
    '''

#主函数
def main():   
    topic_list = []
    hot_zhutiba(topic_list) # 获取主题吧名称 链接
    start = '20180601'  # 设置爬取开始时间
    end = '20180605'  # 设置爬取结束时间,不包含此日

    time1 = time.time()
    for i in range(0,1): # len(topic_list)
        time3 = time.time()
        page = 1  # 财经评论吧 page42-49 即发表于2月27日 + 2月28日 的全部评论 
        # 2016:2525[2550 4]/2482/2435/2360/2264/2179/2109/2038/1980/
        # 2017:1935[start 1]/1887/1828/1769/1703/1644/1559/1419/1137/
        # [2017.10]925/608/323/
        # 2018:152/68/
        topic_bar = topic_list[i][0]
        topic_url = topic_list[i][-1]
        file = "D:\\Program Files\\Git\\git_work\\demo-topic-model\\" + start + "_" + end + "_cjpl.csv"        #设置评论内容的存储路径 
        start_url = "http://guba.eastmoney.com/list," + topic_url + ",99,f_" + str(page) + ".html"
        # http://guba.eastmoney.com/list,cjpl,99_1.html   按评论时间排序
        # http://guba.eastmoney.com/list,cjpl,99,f_1.html 按发帖时间排序
        print("------------------- Start Crawl -------------------")
        with open(file, 'a', newline="", encoding='gb18030') as f:
            csvwriter = csv.writer(f,dialect = ("excel"))
            csvwriter.writerow(["页面信息","评论标题","发表作者","发表时间","阅读量","评论量","内容链接","评论内容"])
        while True: 
            if start_url != "":              
                start_url = getCommentInfo(start_url,file,page,start,end,topic_bar,topic_url)        
                page = page + 1
            else:
                print("已完成topic " + topic_bar + " " + start + "-" + end + "的评论爬取")
                break
        time4 = time.time()
        m, s = divmod(time4-time3, 60)
        h, m = divmod(m, 60)
        print("topic " + topic_bar + " all time using %02d:%02d:%02d" % (h, m, s))
        print("------------------- End   Crawl -------------------")
        time2 = time.time()
        m, s = divmod(time2-time1, 60)
        h, m = divmod(m, 60)
        print ("all time using %02d:%02d:%02d" % (h, m, s))

main()

------------------- Start Crawl -------------------
财经评论吧 p1 1 In and Crawling success,time using 0.60002 s
财经评论吧 p1 2 In and Crawling success,time using 1.03641 s
财经评论吧 p1 3 In and Crawling success,time using 0.81190 s
财经评论吧 p1 4 In and Crawling success,time using 1.02207 s
财经评论吧 p1 5 In and Crawling success,time using 1.63360 s
财经评论吧 p1 6 In and Crawling success,time using 0.62974 s
财经评论吧 p1 7 In and Crawling success,time using 0.87790 s
财经评论吧 p1 8 In and Crawling success,time using 0.64613 s
财经评论吧 p1 9 In and Crawling success,time using 0.45371 s
财经评论吧 p1 10 In and Crawling success,time using 0.86042 s
财经评论吧 p1 11 In and Crawling success,time using 0.76363 s
财经评论吧 p1 12 In and Crawling success,time using 2.93725 s
财经评论吧 p1 13 In and Crawling success,time using 0.83294 s
财经评论吧 p1 14 In and Crawling success,time using 2.10049 s
财经评论吧 p1 15 In and Crawling success,time using 0.72466 s
财经评论吧 p1 16 In and Crawling success,time using 2.36375 s
财经评论吧 p1 17 In and Crawling success,time usin

财经评论吧 p2 65 In and Crawling success,time using 0.76050 s
财经评论吧 p2 66 In and Crawling success,time using 0.68055 s
财经评论吧 p2 67 In and Crawling success,time using 0.68358 s
财经评论吧 p2 68 In and Crawling success,time using 0.47101 s
财经评论吧 p2 69 In and Crawling success,time using 0.49565 s
财经评论吧 p2 70 In and Crawling success,time using 0.66937 s
财经评论吧 p2 71 In and Crawling success,time using 0.54239 s
财经评论吧 p2 72 In and Crawling success,time using 0.60824 s
财经评论吧 p2 73 In and Crawling success,time using 0.71204 s
财经评论吧 p2 74 In and Crawling success,time using 0.51373 s
财经评论吧 p2 75 In and Crawling success,time using 0.52351 s
财经评论吧 p2 76 In and Crawling success,time using 0.84040 s
财经评论吧 p2 77 In and Crawling success,time using 0.85825 s
财经评论吧 p2 78 In and Crawling success,time using 0.71304 s
财经评论吧 p2 79 In and Crawling success,time using 0.64538 s
财经评论吧 p2 80 In and Crawling success,time using 2.17092 s
财经评论吧 p3 1 In and Crawling success,time using 1.12147 s
财经评论吧 p3 2 In and Crawling succe

财经评论吧 p4 50 In and Crawling success,time using 0.76994 s
财经评论吧 p4 51 In and Crawling success,time using 0.88144 s
财经评论吧 p4 52 In and Crawling success,time using 0.85111 s
财经评论吧 p4 53 In and Crawling success,time using 0.51077 s
财经评论吧 p4 54 In and Crawling success,time using 1.77116 s
财经评论吧 p4 55 In and Crawling success,time using 0.58594 s
财经评论吧 p4 56 In and Crawling success,time using 0.61408 s
财经评论吧 p4 57 In and Crawling success,time using 0.79445 s
财经评论吧 p4 58 In and Crawling success,time using 0.88341 s
财经评论吧 p4 59 In and Crawling success,time using 0.50241 s
财经评论吧 p4 60 In and Crawling success,time using 0.55371 s
财经评论吧 p4 61 In and Crawling success,time using 0.81164 s
财经评论吧 p4 62 In and Crawling success,time using 1.57786 s
财经评论吧 p4 63 In and Crawling success,time using 0.71401 s
财经评论吧 p4 64 In and Crawling success,time using 0.66993 s
财经评论吧 p4 65 In and Crawling success,time using 0.56858 s
财经评论吧 p4 66 In and Crawling success,time using 0.46050 s
财经评论吧 p4 67 In and Crawling suc

财经评论吧 p6 35 Out
财经评论吧 p6 36 Out
财经评论吧 p6 37 Out
财经评论吧 p6 38 Out
财经评论吧 p6 39 Out
财经评论吧 p6 40 Out
财经评论吧 p6 41 Out
财经评论吧 p6 42 Out
财经评论吧 p6 43 Out
财经评论吧 p6 44 Out
财经评论吧 p6 45 Out
财经评论吧 p6 46 Out
财经评论吧 p6 47 Out
财经评论吧 p6 48 Out
财经评论吧 p6 49 Out
财经评论吧 p6 50 Out
财经评论吧 p6 51 Out
财经评论吧 p6 52 Out
财经评论吧 p6 53 Out
财经评论吧 p6 54 Out
财经评论吧 p6 55 Out
财经评论吧 p6 56 Out
财经评论吧 p6 57 Out
财经评论吧 p6 58 Out
财经评论吧 p6 59 Out
财经评论吧 p6 60 Out
财经评论吧 p6 61 Out
财经评论吧 p6 62 Out
财经评论吧 p6 63 Out
财经评论吧 p6 64 Out
财经评论吧 p6 65 Out
财经评论吧 p6 66 Out
财经评论吧 p6 67 Out
财经评论吧 p6 68 Out
财经评论吧 p6 69 Out
财经评论吧 p6 70 Out
财经评论吧 p6 71 Out
财经评论吧 p6 72 Out
财经评论吧 p6 73 Out
财经评论吧 p6 74 Out
财经评论吧 p6 75 Out
财经评论吧 p6 76 Out
财经评论吧 p6 77 Out
财经评论吧 p6 78 Out
财经评论吧 p6 79 Out
财经评论吧 p6 80 Out
已完成topic 财经评论吧 20180601-20180605的评论爬取
topic 财经评论吧 all time using 00:06:57
------------------- End   Crawl -------------------
all time using 00:06:57
